<a href="https://colab.research.google.com/github/roysaurav/US-GDP-Model/blob/main/US_GDP_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# **Setup**




In [ ]:
!pip install plotly==4.8

     |████████████████████████████████| 11.5MB 266kB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [ ]:
Forward_Looking_Months = 1 #@param {type:"integer"}
# Change training end date for out of sample test
model_training_start_date = '2015-01-01' #@param {type:"date"}
model_training_end_date = '2020-12-31' #@param {type:"date"}
model_testing_start_date = '2020-01-01' #@param {type:"date"}
model_testing_end_date = '2020-12-31' #@param {type:"date"}

In [ ]:
'''
from google.colab import drive
drive.mount("/content/drive")
'''

'\nfrom google.colab import drive\ndrive.mount("/content/drive")\n'

In [ ]:
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import plotly.express as px
from sklearn.metrics import mean_squared_error, r2_score

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
#@title Data Prep
dep_var = "US Nominal GDP"	
path = "/content/drive/MyDrive/CI Global/Project.xlsx"
path  = "https://drive.google.com/uc?export=download&id=16hS_2LyULBmsjLd8cCLkRJ-2WJA1U082"
df = pd.read_excel(path, sheet_name="US GDP Data", skiprows=1).set_index("Date").sort_index().dropna()
df[dep_var] = df[dep_var].shift(-Forward_Looking_Months)
df.head()

,US Nominal GDP,US Real GDP,Composite Index of Leading Indicators,Conference Board US Leading,PMI Output,PMI New Orders,US Manufacturing PMI stocks,US Service PMI Business activity,US Composite PMI,disposable income as % of saving,US Retail Sales,average hourly earning,Consumer Price Index (CPI),OECD Employment,US Unemployment Rate,US Employees on Nonfarm Payrolls,US Initial Jobless Claims SA,New Privately Owned Housing Units,US Real GDP Quantity Residenti
Date,,,,,,,,,,,,,,,,,,,
2009-10-30,-2.8,-0.3,98.0037,-6.5,57.8,54.1,45.7,54.7,55.2,5.4,-3.2,2.5,-3.0,79.08,9.63,-199,522,534,92.143
2009-11-30,0.5,-0.3,98.4397,-2.8,58.5,59.1,47.7,55.4,55.9,5.9,1.5,2.2,-2.0,79.08,9.63,12,475,588,92.143
2009-12-31,0.5,-2.8,98.8393,1.3,58.3,58.4,45.5,57.5,57.7,5.9,4.2,1.9,-0.3,75.77,9.93,-269,468,581,91.952
2010-01-29,0.5,-2.8,99.1850,3.9,57.2,58.0,48.4,57.5,57.5,6.1,2.7,2.0,1.5,75.77,9.93,2,496,614,91.952
2010-02-26,2.3,-2.8,99.4448,5.4,57.9,57.5,50.3,59.1,58.9,5.8,3.3,2.1,2.4,75.77,9.93,-92,488,604,91.952


In [ ]:
vars = list(df.columns)
vars.remove(dep_var)

# **Correlation**


In [ ]:
#@title Full Series Correlations

corr_ = df.loc[model_training_start_date:].corr()
fig = px.imshow(corr_, x= df.columns, y=df.columns, color_continuous_scale="RdYlGn", width=700, height=700)
fig.show()
#sns.heatmap(corr_)

In [ ]:
rolling_corr_df = df.rolling(12*3).corr().dropna().reset_index()
rolling_corr_df = rolling_corr_df[rolling_corr_df["level_1"] == dep_var]
rolling_corr_df = rolling_corr_df.drop(columns=[dep_var, "level_1"]).set_index("Date")


In [ ]:
#@title Real GDP Correlation
fig = rolling_corr_df[["US Real GDP", "US Real GDP Quantity Residenti"]].plot(backend="plotly", title="GDP Correlation")
fig.update_yaxes(title_text='Correlation')
fig.show()

In [ ]:
#@title Leading Indicator Correlation

fig = rolling_corr_df[["Composite Index of Leading Indicators", "Conference Board US Leading"]].plot(backend="plotly")
fig.update_yaxes(title_text='Correlation')
fig.show()

In [ ]:
#@title PMI Indicator Correlation

fig = rolling_corr_df[["PMI Output", "PMI New Orders", "US Manufacturing PMI stocks", "US Service PMI Business activity", "US Composite PMI"]].plot(backend="plotly")
fig.update_yaxes(title_text='Correlation')
fig.show()

In [ ]:
#@title Wages, Savings and Prices Correlation


fig = rolling_corr_df[["disposable income as % of saving", "US Retail Sales", "average hourly earning", "Consumer Price Index (CPI)"]].plot(backend="plotly")
fig.update_yaxes(title_text='Correlation')
fig.show()

In [ ]:
#@title Housing Metrics Correlation

fig = rolling_corr_df[["New Privately Owned Housing Units "]].plot(backend="plotly")
fig.update_yaxes(title_text='Correlation')
fig.show()

In [ ]:
#@title Employment Metrics Correlation

fig = rolling_corr_df[["OECD Employment", "US Unemployment Rate", "US Employees on Nonfarm Payrolls", "US Initial Jobless Claims SA"]].plot(backend="plotly")
fig.update_yaxes(title_text='Correlation')
fig.show()

# **Model**


In [ ]:
def run_model(var_):
  X = df_[var_]
  Y = df_[[dep_var]]
  X = sm.add_constant(X) # adding a constant

  mod = sm.OLS(Y,X)
  res = mod.fit()
  return res.rsquared, res.rsquared_adj, res.pvalues[var_[-1]]



In [ ]:
dep_var = "US Nominal GDP"	
vars = list(df.columns)
vars.remove(dep_var)

In [ ]:
#@title Scaling Variables for Model

df = (df - df.min())/(df.min() - df.max())
df = df.dropna()
df.tail()

,US Nominal GDP,US Real GDP,Composite Index of Leading Indicators,Conference Board US Leading,PMI Output,PMI New Orders,US Manufacturing PMI stocks,US Service PMI Business activity,US Composite PMI,disposable income as % of saving,US Retail Sales,average hourly earning,Consumer Price Index (CPI),OECD Employment,US Unemployment Rate,US Employees on Nonfarm Payrolls,US Initial Jobless Claims SA,New Privately Owned Housing Units,US Real GDP Quantity Residenti
Date,,,,,,,,,,,,,,,,,,,
2020-04-30,-0.726027,-0.847458,-0.000000,-0.000000,-0.000000,-0.000000,-0.110390,-0.000000,-0.000000,-1.000000,-0.000000,-1.000000,-1.000000,-1.0,-0.021119,-0.000000,-0.549955,-0.379091,-0.971636
2020-05-29,-0.000000,-0.847458,-0.184653,-0.120536,-0.170732,-0.197708,-0.000000,-0.314869,-0.294118,-0.681979,-0.508897,-0.772727,-0.728324,-1.0,-0.021119,-0.921136,-0.254425,-0.473636,-0.971636
2020-06-30,-0.000000,-0.847458,-0.381341,-0.245536,-0.570122,-0.638968,-0.538961,-0.618076,-0.614706,-0.480565,-0.786477,-0.515152,-0.647399,-1.0,-1.000000,-1.000000,-0.181068,-0.680000,-0.732011
2020-07-31,-0.000000,-0.847458,-0.547267,-0.312500,-0.698171,-0.676218,-0.675325,-0.679300,-0.685294,-0.459364,-0.804270,-0.469697,-0.687861,-1.0,-1.000000,-0.877767,-0.148515,-0.881818,-0.732011
2020-08-31,-0.465753,-0.847458,-0.604662,-0.375000,-0.728659,-0.756447,-0.837662,-0.825073,-0.811765,-0.325088,-0.836299,-0.469697,-0.485549,-1.0,-1.000000,-0.872165,-0.102460,-0.778182,-0.732011


In [ ]:
#@title Searching Best GDP Model

# raise when error when found best GDP Model
required_var = []
adjusted_rsquares = []
temp_rsquares = pd.DataFrame(data={"rsquared_adj":[],"rsquared":[], "name":[], "p_values":[] })
result_df = pd.DataFrame(data={"rsquared_adj":[],"rsquared":[], "num_vars" :[]})
df_ = df.loc[model_training_start_date:model_training_end_date]
while vars != []:
  for var in vars:
    rsquared, rsquared_adj, pvalues = run_model(required_var + [var])
    temp_rsquares_ = pd.DataFrame(data={"rsquared_adj":[rsquared_adj],"rsquared":[rsquared], "name":[var], "p_values":[pvalues] })
    temp_rsquares = pd.concat([temp_rsquares, temp_rsquares_])
  try:
    temp_rsquares = temp_rsquares[abs(temp_rsquares["p_values"]) < 0.05]
    new_var = temp_rsquares[temp_rsquares["rsquared"] == max(temp_rsquares["rsquared"])]["name"].iloc[0]
    required_var.append(new_var)
    vars.remove(new_var)
    temp_rsquares_ = temp_rsquares_.drop(columns=["name"])
    temp_rsquares_["num_vars"] = len(required_var)
    result_df = pd.concat([result_df, temp_rsquares_])
    temp_rsquares = pd.DataFrame(data={"rsquared_adj":[],"rsquared":[], "name":[] , "p_values":[]})
  except:
    print("Best Model Found")
    vars = []

Best Model Found


In [ ]:
fig = result_df.plot(x ="num_vars", y ="rsquared_adj", backend="plotly")
fig.update_yaxes(title_text='Model fit Adjusted for Number of Variables')
fig.update_xaxes(title_text='Number of Variables')

fig.show()

In [ ]:
#@title Best GDP Model Parameters

X = df_[required_var]
Y = df_[[dep_var]]
X = sm.add_constant(X) # adding a constant

mod = sm.OLS(Y,X)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:         US Nominal GDP   R-squared:                       0.966
Model:                            OLS   Adj. R-squared:                  0.961
Method:                 Least Squares   F-statistic:                     209.1
Date:                Fri, 19 Mar 2021   Prob (F-statistic):           2.38e-40
Time:                        18:44:09   Log-Likelihood:                 129.76
No. Observations:                  68   AIC:                            -241.5
Df Residuals:                      59   BIC:                            -221.5
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                                            coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------

In [ ]:
#@title Best GDP Model Test - Mean Square Error 

# In sample test
# Change training end date for out of sample test

pred_x = df.loc[model_testing_start_date:model_testing_end_date][required_var]

# Add a constant for training
pred_x["const"] = 1.0 
cols = pred_x.columns.tolist()
cols = [cols[-1]]+cols[:-1]
pred_x = pred_x[cols]

pred_y = res.predict(pred_x)
actutal_y = df.loc[model_testing_start_date:model_testing_end_date][dep_var]
print('Mean Squared Error: %.2f' % mean_squared_error(actutal_y, pred_y))

Mean Squared Error: 0.01
